In [15]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-194855
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-194855


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "bogicluster1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    my_cluster=ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster. Use it.")
except:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    my_cluster=ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config )
    my_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
import os
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-for-project1/code/Users/odl_user_194855/udacity_azure_project1-master/udacity_azure_project1-master'

In [45]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(parameter_space={'penalty': choice('l1','l2','elasticnet','none'), 'C': uniform(0.2,2)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-for-project1/code/Users/odl_user_194855/udacity_azure_project1-master/udacity_azure_project1-master',
    script='./train.py',
    compute_target=my_cluster,
    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(hyperparameter_sampling=ps,primary_metric_name="accuracy",primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
policy=policy,run_config=src, max_total_runs=5)

In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment=Experiment(ws,'myexperiment')
hyperdrive_run=experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
import joblib
# Get your best run and save the model from that run.

best_run=hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(best_run,'model.joblib')

['model.joblib']

In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [37]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50)

In [40]:
x_train['target_var']=y_train


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= x_train,
    label_column_name='target_var',
    n_cross_validations= 5)

In [44]:
# Submit your automl run

experiment_automl=Experiment(ws,'AutoMl_experiment')
run_automl=experiment_automl.submit(config=automl_config,show_output=True)

Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoMl_experiment,AutoML_611a38cd-f1fb-4a0b-9095-f8305864a80b,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


KeyboardInterrupt: 

In [ ]:
# Retrieve and save your best automl model.

best_run,fitted_model=run_automl.get_output()
joblib.dump(fitted_model,'automml_model.joblib')
